In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import typing as t
from arcgis.features import GeoAccessor, FeatureLayer, FeatureLayerCollection
from arcgis.gis import GIS
import arcpy
pgis = GIS("pro")
from pathlib import Path
import os

In [3]:
# Get info from hosted feature
#hostedfeature_ID = '615975bee41841c9bc74d863751400c4#data'
hostedfeature_ID = 'e5c5395588e045ea83f2cf07c76ed7f7'


featuresserver_hostedfeature = pgis.content.get(hostedfeature_ID)
print(featuresserver_hostedfeature)
featurelayer_hostedfeature = featuresserver_hostedfeature.layers[0]
featurelayer_features = featurelayer_hostedfeature.query().features

<Item title:"Boringer_090323" type:Feature Layer Collection owner:jdr@multiconsult.no>


In [12]:
img = r'\\nsv2-nasuni-02\fredrikstad\Prosjekt\O10245\10245026-01\10245026-01-03_ARBEIDSOMRAADE\21_fagomraade\11_Geoteknikk\10245026-03-TEGNINGER\GeoLab\images\10245026-02-RIG-TEG-SB-2041-200_rev00.png'

fset = featurelayer_hostedfeature.query()
fset.features
featurelayer_hostedfeature.attachments.get_list(oid=21)
#featurelayer_hostedfeature.attachments.add(21, img)


[{'data_size': 846408,
  'exifInfo': None,
  'size': 846408,
  'content_type': 'image/png',
  'keywords': '',
  'name': '10245026-02-RIG-TEG-SB-2041-200_rev00.png',
  'globalid': '{4AC4F9DC-A71E-48D0-801E-04E4F0B2FF9C}',
  'id': 84,
  'attachmentid': 84,
  'att_name': '10245026-02-RIG-TEG-SB-2041-200_rev00.png',
  'contentType': 'image/png'},
 {'data_size': 120264,
  'exifInfo': None,
  'size': 120264,
  'content_type': 'image/png',
  'keywords': '',
  'name': 'SB-2040.png',
  'globalid': '{C919580D-A0CF-435F-AF15-0FE5B9106145}',
  'id': 21,
  'attachmentid': 21,
  'att_name': 'SB-2040.png',
  'contentType': 'image/png'},
 {'data_size': 641137,
  'exifInfo': None,
  'size': 641137,
  'content_type': 'image/png',
  'keywords': '',
  'name': 'SB-2040_PR1.png',
  'globalid': '{67D21C50-E8E5-4792-937A-BB271EC41DC3}',
  'id': 82,
  'attachmentid': 82,
  'att_name': 'SB-2040_PR1.png',
  'contentType': 'image/png'},
 {'data_size': 846408,
  'exifInfo': None,
  'size': 846408,
  'content_type'

In [12]:
# Functions
def add_picture(input_fc, inputField, pathField):
    # The input feature class must first be GDB attachments enabled
    arcpy.EnableAttachments_management(input_fc)
    # Use the match table with the Add Attachments tool
    arcpy.AddAttachments_management(input_fc, inputField, input_fc, inputField, 
                                    pathField, None)
    
def remove_word(sentence, word):
    """
        Input:
            sentence: string
            word: word to remove
        Output:
            returns sentence without word
    """
    if isinstance(sentence, str):
        return sentence.replace(word, '').strip()
    else:
        return None

def tolket(row):
    val = ''
    if isinstance(row['Metode'], str):        
        if 'Tolk' in row['Metode']:
            val = 'Tolk'
    return val

def bergkote(row):
    if row['Stopp'] in [93,94]:
        val = str(round(row['Z'] - row['Løsm'], 2))
    else:
        val = '~'
    return val

def z_kom(row):
    if row['Z'] == 0:
        val = 'Missing Z'
    else:
        val = ''
    return val

def bilde(row, url):
    val = url + r'\images' + "\\" + row['Borhull'] + '.png'
    arcpy.AddMessage(val)
    return val


def df_from_geosuite(xl_file: str, pdf_folder: str, crs) -> pd.DataFrame:
    cols = ['Borhull', 'X', 'Y', 'Z', 'Metode', 'Stopp', 'Løsm', 'Fjell']

    if xl_file.endswith('.xls'):
        df = pd.read_html(xl_file, decimal=',', thousands=None)[0][cols]
    else:
        df = pd.read_excel(xl_file, engine='openpyxl', usecols=cols)

    df['Borhull'] = df['Borhull'].astype("string")
    arcpy.AddMessage(df.dtypes)
    df['Tolket'] = df.apply(tolket, axis=1)
    df['Metode'] = df['Metode'].apply(remove_word, word='Tolk')
    df['Bergkote'] = df.apply(bergkote, axis=1)
    df['kommentar'] = df.apply(z_kom, axis=1)
    df['Bilde'] = df.apply(bilde, url=pdf_folder, axis=1)
    df.columns = [x.lower() for x in df.columns]

    #print(df.head())

    # convert df to spatially enabled dataframe
    sedf = pd.DataFrame.spatial.from_xy(df, "y", "x", sr=crs)  
    # notice the X and Y positions
    return df #sedf

In [15]:
xl_file = r'\\nsv2-nasuni-02\fredrikstad\Prosjekt\O10245\10245026-01\10245026-01-03_ARBEIDSOMRAADE\21_fagomraade\11_Geoteknikk\10245026-10-GEOSUITE\Supplerende GRUS 2023\Test2.xls'
pdf_folder = r'\\nsv2-nasuni-02\fredrikstad\Prosjekt\O10245\10245026-01\10245026-01-03_ARBEIDSOMRAADE\21_fagomraade\11_Geoteknikk\10245026-10-GEOSUITE\Supplerende GRUS 2023\AUTOGRAF.RIT'
crs = 5111
df = df_from_geosuite(xl_file, pdf_folder, crs)
#dfseries = df.squeeze()

In [16]:
df

,borhull,x,y,z,metode,stopp,løsm,fjell,tolket,bergkote,kommentar,bilde,SHAPE
0,SB-2001,1142289.501,79222.189,31.802,Total,93,36.41,0.00,,-4.61,,\\nsv2-nasuni-02\fredrikstad\Prosjekt\O10245\1...,"{""spatialReference"": {""wkid"": 5111}, ""x"": 7922..."
1,SB-2002,1142293.917,79478.880,25.428,Total,90,17.12,NaN,,~,,\\nsv2-nasuni-02\fredrikstad\Prosjekt\O10245\1...,"{""spatialReference"": {""wkid"": 5111}, ""x"": 7947..."
2,SB-2003,1142273.448,79194.757,31.360,Total,90,57.40,NaN,,~,,\\nsv2-nasuni-02\fredrikstad\Prosjekt\O10245\1...,"{""spatialReference"": {""wkid"": 5111}, ""x"": 7919..."
3,SB-2003A,1142273.448,79194.757,31.360,Total,94,32.28,3.01,Tolk,-0.92,,\\nsv2-nasuni-02\fredrikstad\Prosjekt\O10245\1...,"{""spatialReference"": {""wkid"": 5111}, ""x"": 7919..."
4,SB-2004,1142271.376,79579.766,34.980,Total,94,6.48,3.00,Tolk,28.5,,\\nsv2-nasuni-02\fredrikstad\Prosjekt\O10245\1...,"{""spatialReference"": {""wkid"": 5111}, ""x"": 7957..."
5,SB-2005,1142280.454,79235.480,31.876,Total,90,25.02,NaN,,~,,\\nsv2-nasuni-02\fredrikstad\Prosjekt\O10245\1...,"{""spatialReference"": {""wkid"": 5111}, ""x"": 7923..."
6,SB-2006,1142270.467,79394.461,26.417,Total,90,25.35,NaN,,~,,\\nsv2-nasuni-02\fredrikstad\Prosjekt\O10245\1...,"{""spatialReference"": {""wkid"": 5111}, ""x"": 7939..."
7,SB-2007,1142264.590,79350.305,26.628,Total Cpt Prøve,92,34.90,NaN,,~,,\\nsv2-nasuni-02\fredrikstad\Prosjekt\O10245\1...,"{""spatialReference"": {""wkid"": 5111}, ""x"": 7935..."
8,SB-2008,1142264.042,79655.463,34.639,Total Cpt,94,6.29,2.70,Tolk,28.35,,\\nsv2-nasuni-02\fredrikstad\Prosjekt\O10245\1...,"{""spatialReference"": {""wkid"": 5111}, ""x"": 7965..."
9,SB-2009,1142253.674,79735.895,40.704,Total,90,1.99,NaN,,~,,\\nsv2-nasuni-02\fredrikstad\Prosjekt\O10245\1...,"{""spatialReference"": {""wkid"": 5111}, ""x"": 7973..."


In [17]:
# Add dataframe from excel to hosted feature. Might need to remove the dupicates
featurelayer_hostedfeature.edit_features(adds=df)

{'addResults': [{'objectId': 109,
   'uniqueId': 109,
   'globalId': 'B0BE0398-1131-41FB-AAE5-F6D9FCC7EF37',
   'success': True},
  {'objectId': 110,
   'uniqueId': 110,
   'globalId': '7DDC36D6-EF6C-4509-AA36-DB8D7A222D76',
   'success': True},
  {'objectId': 111,
   'uniqueId': 111,
   'globalId': '432C261A-EFC4-424C-971A-B6199EF1565A',
   'success': True},
  {'objectId': 112,
   'uniqueId': 112,
   'globalId': 'E4630427-3B6E-4318-8454-234754DD4B11',
   'success': True},
  {'objectId': 113,
   'uniqueId': 113,
   'globalId': '2784F0D7-3EFA-48EC-B562-B4415EFCCBD2',
   'success': True},
  {'objectId': 114,
   'uniqueId': 114,
   'globalId': '55FBA1D7-F3F1-413A-977D-52F36FF82B37',
   'success': True},
  {'objectId': 115,
   'uniqueId': 115,
   'globalId': 'B4508EDC-62D4-4D03-84C0-AC48B282AA5D',
   'success': True},
  {'objectId': 116,
   'uniqueId': 116,
   'globalId': '7902EC4F-B77C-45B7-9984-5E8E8E71798C',
   'success': True},
  {'objectId': 117,
   'uniqueId': 117,
   'globalId': '46

In [39]:
# Make a list of dict with file name and attachment
def img_folder_items(png_folder) -> list:
    '''
    Returns a list of dictionaries that contains which profiles have new pictures
    '''
    item_name_and_attachment = []
    for item in os.listdir(png_folder):
        arcpy.AddMessage(item)
        if item.split('.')[-1] == 'png':
            # Find file name
            file_name = item.split('.')[0]
            file_dict = {
                'file_name': file_name,
                'attachment': png_folder+item
                }
            item_name_and_attachment.append(file_dict)
    return item_name_and_attachment


# Returns a list of objectid of which bps that is going to be added and the attachments
def get_objectid(fl_hf, item_name_and_attachment):
    fl_features = fl_hf.query().features
    objectsandattachments = []
    for item in item_name_and_attachment:
        image_name, attachment = item.get('file_name'), item.get('attachment')
        arcpy.AddMessage(f'Finner profil {image_name}')
        # try:
        filtered_features = [f for f in fl_features if f.attributes['borhull'] == image_name]
        # except Exception as e:
            #print(f'Sammensvarer profil navn og nummer på bildet med linjenavnet i GIS? Bildet skal hete sonenr_sonenavn_profilnr, {type(e)}')
        if filtered_features:
            edit_feature = filtered_features[0]
        else:
            # handle the case where no matching feature was found
            edit_feature = None  # or some other default value

        if edit_feature is not None:
            print(edit_feature.attributes.get('OBJECTID'))
            oid = edit_feature.attributes.get('OBJECTID')

            objectsandattachmentdict = {
                'objectid': oid,
                'attachment': attachment
                            }
            objectsandattachments.append(objectsandattachmentdict)
    return objectsandattachments

def add_attachments(feature_layer, objectsandattachments):
    arcpy.ResetProgressor()
    arcpy.SetProgressor('step', 'Processing features...', 0, len(objectsandattachments), 1)
    a_counter = 0
    #print(objectids[0], objectids[1])
    for odict in objectsandattachments:
        oid = odict.get('objectid')
        print(odict)
        attachment = odict.get('attachment')

        existing_attachments = feature_layer.attachments.get_list(oid=oid)
        bp_name = attachment.split(r'\\')[-1]
        if not existing_attachments:
            feature_layer.attachments.add(oid, attachment)
            print(attachment)
            arcpy.AddMessage(f'Added attachment {attachment} to bp {bp_name}')
            a_counter += 1
        else:
            arcpy.AddMessage(f'No new attachements added for id {oid} bp name {bp_name}')
            
        arcpy.AddMessage(f'Attachments added: {a_counter}')

In [40]:
png_folder = r'\\nsv2-nasuni-02\fredrikstad\Prosjekt\O10245\10245026-01\10245026-01-03_ARBEIDSOMRAADE\21_fagomraade\11_Geoteknikk\10245026-10-GEOSUITE\Supplerende GRUS 2023\AUTOGRAF.RIT\images'

items = img_folder_items(png_folder)
print(items)
objectsandattachments = get_objectid(featurelayer_hostedfeature, items)
add_attachments(featurelayer_hostedfeature, objectsandattachments)

[{'file_name': 'SB-2021', 'attachment': '\\\\nsv2-nasuni-02\\fredrikstad\\Prosjekt\\O10245\\10245026-01\\10245026-01-03_ARBEIDSOMRAADE\\21_fagomraade\\11_Geoteknikk\\10245026-10-GEOSUITE\\Supplerende GRUS 2023\\AUTOGRAF.RIT\\imagesSB-2021.png'}, {'file_name': 'SB-2013', 'attachment': '\\\\nsv2-nasuni-02\\fredrikstad\\Prosjekt\\O10245\\10245026-01\\10245026-01-03_ARBEIDSOMRAADE\\21_fagomraade\\11_Geoteknikk\\10245026-10-GEOSUITE\\Supplerende GRUS 2023\\AUTOGRAF.RIT\\imagesSB-2013.png'}, {'file_name': 'SB-2014', 'attachment': '\\\\nsv2-nasuni-02\\fredrikstad\\Prosjekt\\O10245\\10245026-01\\10245026-01-03_ARBEIDSOMRAADE\\21_fagomraade\\11_Geoteknikk\\10245026-10-GEOSUITE\\Supplerende GRUS 2023\\AUTOGRAF.RIT\\imagesSB-2014.png'}, {'file_name': 'SB-2026', 'attachment': '\\\\nsv2-nasuni-02\\fredrikstad\\Prosjekt\\O10245\\10245026-01\\10245026-01-03_ARBEIDSOMRAADE\\21_fagomraade\\11_Geoteknikk\\10245026-10-GEOSUITE\\Supplerende GRUS 2023\\AUTOGRAF.RIT\\imagesSB-2026.png'}, {'file_name': 'SB-2

In [43]:
featurelayer_url = r'https://services-eu1.arcgis.com/duOysKJu8BseU0Gy/arcgis/rest/services/SF_MC_3FM_RIG_Befaring/FeatureServer/2'
featurelayer_source = FeatureLayer(featurelayer_url)

#featurelayer_source.attachments.download
bilder = featurelayer_source.query()

folder = r'C:\Users\jdr\OneDrive - Multiconsult\Skrivebord\Totsonderinger\TestToolBilder'

for bilde in bilder:
    object_id = bilde.attributes.get('OBJECTID')
    bilde.attributes["name"] = 'test.jpg'
    print(bilde.attributes.get('name'))
    featurelayer_source.attachments.get_list(object_id)[0]['name'] = 'test1.jpg'
    
    x_koord = bilde.attributes['x_koordinat']
    y_koord = bilde.attributes['y_koordinat']
    update_dict = {'name': 'test2.jpeg'}
    print(featurelayer_source.properties())
    featurelayer_source.manager.update_definition(update_dict)
    featurelayer_source.attachments.download(attachment_id=object_id, save_path=folder,)

    print(bilde.attributes['x_koordinat'])#attachments.get_list(object_id)[0])

#print(featurelayer_source.attachments.get_list(1))

test.jpg


TypeError: __call__() missing 1 required positional argument: 'key'